In [1]:
import os
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import pandas as pd
import csv
import time
import sqlite3

In [ ]:
#get all product
driver = webdriver.Chrome('chromedriver')
driver.implicitly_wait(30)
driver.maximize_window()
driver.get('https://www.tokopedia.com/gudangjaket/page/2')
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
time.sleep(3)

elems = driver.find_elements_by_xpath("//a")
data = []
for elem in elems:
    
    data_test = elem.get_attribute("data-testid")
    if data_test == "lnkProductContainer":
        href = elem.get_attribute("href")
        product_name = elem.text
        product_name = product_name.replace('Produk Unggulan\n','')
        product_name = product_name.replace('Stok Kosong\n','')
        products = product_name.split('\n')
        row = [href,products[0],products[1]]
        data.append(row)

driver.quit()
df = pd.DataFrame(data,columns=['href','name','price'])
# df.head(n=10)
# df.to_csv(r'toped_master_data.csv', index = False, header=True, doublequote=True)
conn = sqlite3.connect('scrap.gudangjaket.db')
df.to_sql('GUDANGJAKET_MASTER', conn, if_exists='append', index = False)

In [4]:
#grab detail
conn = sqlite3.connect('scrap.gudangjaket.db')
cur = conn.cursor()
cur.execute('select a.href, a.name from GUDANGJAKET_MASTER a left join GUDANGJAKET_IMAGES b on a.name=b.name where b.name is null limit 1')
reader = cur.fetchall()
data_images = []
data_desc = []
data_weight = []
data_variant = []
for line in reader:
    # print(line[0])
    chrome_options = webdriver.ChromeOptions()
    prefs = {"profile.default_content_setting_values.notifications" : 2}
    chrome_options.add_experimental_option("prefs",prefs)
#     driver = webdriver.Chrome(chrome_options=chrome_options)
    driver = webdriver.Chrome('chromedriver',chrome_options=chrome_options)
    driver.implicitly_wait(30)
    driver.maximize_window()
    driver.get(line[0])
    
    time.sleep(2)

    # #fetch thumbnail
    # elems = driver.find_elements_by_xpath("//img")
    # for elem in elems:
    #     data_test = elem.get_attribute("alt")
    #     if data_test == "Thumbnail":
    #         product_images = elem.get_attribute("src")
    #         if product_images.startswith('https'):        
    #             row = [line[1], product_images]
    #             data_images.append(row)

    # #fetch variant
    # elems = driver.find_elements_by_xpath("//div")
    # for elem in elems:
    #     data_test = elem.get_attribute("data-testid")
    #     if data_test == "lblPDPProductVariantukuran":
    #         row = [line[1], elem.text]
    #         data_variant.append(row)
    #     if data_test == "lblPDPProductVariantwarna":
    #         row = [line[1], elem.text]
    #         data_variant.append(row)

    # #fetch description and weight
    # driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    # time.sleep(2)
    # elems = driver.find_elements_by_xpath("//p")
    # for elem in elems:
    #     data_test = elem.get_attribute("data-testid")
    #     if data_test == "lblPDPDeskripsiProduk":
    #         product_desc = elem.text
    #         row = [line[1], product_desc]
    #         data_desc.append(row)
    #     if data_test == "PDPDetailWeightValue":
    #         product_weight = elem.text
    #         row = [line[1], product_weight]
    #         data_weight.append(row)
    elems = driver.find_elements_by_xpath('//div[@data-testid="PDPImageThumbnail"]')
    for elem in elems:
#         data_test = elem.get_attribute("data-testid")
#         if data_test == "PDPImageThumbnail":
        elem.click()
        time.sleep(1)
        elems = driver.find_element_by_xpath('//img[@alt="product image"]')
        product_image = elems.get_attribute("src")
#         for elems in elemss:
#             data_test = elems.get_attribute("alt")
#             if data_test == "product image":
#                 product_image = elems.get_attribute("src")
        print(product_image)



    driver.quit()
    time.sleep(2)

#insert images
# df = pd.DataFrame(data_images,columns=['name','images'])
# conn = sqlite3.connect('scrap.gudangjaket.db')
# df.to_sql('GUDANGJAKET_IMAGES', conn, if_exists='replace', index = False)

# #insert description
# df = pd.DataFrame(data_desc,columns=['name','images'])
# conn = sqlite3.connect('scrap.gudangjaket.db')
# df.to_sql('GUDANGJAKET_DESC', conn, if_exists='append', index = False)

# #insert weight
# df = pd.DataFrame(data_weight,columns=['name','images'])
# conn = sqlite3.connect('scrap.gudangjaket.db')
# df.to_sql('GUDANGJAKET_WEIGHT', conn, if_exists='append', index = False)

# #insert variant
# df = pd.DataFrame(data_variant,columns=['name','images'])
# conn = sqlite3.connect('scrap.gudangjaket.db')
# df.to_sql('GUDANGJAKET_VARIANT', conn, if_exists='append', index = False)


https://ecs7.tokopedia.net/img/cache/700/product-1/2020/4/18/1618811/1618811_20c23047-55c3-4d44-9e6d-887f89dc32c9_1080_1080.webp
https://ecs7.tokopedia.net/img/cache/700/product-1/2019/11/28/1618811/1618811_9bb444f7-9341-4cae-b1e8-94e7b5934c2a_1145_1145.webp
https://ecs7.tokopedia.net/img/cache/700/product-1/2020/1/7/1618811/1618811_eed2a8f2-f9ed-4ec2-9327-353b9cf6f878_1080_1080.webp
https://ecs7.tokopedia.net/img/cache/700/product-1/2020/5/5/1618811/1618811_65f65b88-ba93-48af-85b0-dc34c083cbbf_1280_1280.webp


ElementClickInterceptedException: Message: element click intercepted: Element <div role="button" data-testid="PDPImageThumbnail" class="css-1muhp5u ejaoon00">...</div> is not clickable at point (760, 695). Other element would receive the click: <div id="chrome_desktop_backend" style="z-index: 999999999;position: fixed;height: 100%;top: 0px;left: 0px;width: 100%;background: rgba(0, 0, 0, 0.89)">...</div>
  (Session info: chrome=81.0.4044.138)
